In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

RAG - Reasoning Engine with Vertex AI Search
@forusone (shins777@gmail.com)


### Reasoning Engine in Vertex AI Search

### Install Vertex AI SDK for Python

In [1]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    langchain-google-community \
    google-cloud-discoveryengine \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.3/94.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00


In [2]:
import sys
from IPython.display import Markdown, display

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### Package import and initialize model

In [ ]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Tool,
)

# Preview
from vertexai.preview import (
    rag,
    reasoning_engines)

from vertexai.preview.generative_models import (
    grounding)

# model initialization
vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)
model = GenerativeModel(MODEL)

### Configuration

In [ ]:
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://reasoning_7424"
MODEL = "gemini-1.5-flash"

## RAG Engine Unit Test

### Helper methods

In [ ]:
def create_rag_corpus(display_name:str, path:list):

  # Create RagCorpus
  embedding_model_config = rag.EmbeddingModelConfig(
      publisher_model="publishers/google/models/text-multilingual-embedding-002"
  )

  rag_corpus = rag.create_corpus(
      display_name=display_name,
      embedding_model_config=embedding_model_config,
  )

  # Import Files to the RagCorpus
  rag.import_files(
      rag_corpus.name,
      paths,
      use_advanced_pdf_parsing=True,
      chunk_size=512,  # Optional
      chunk_overlap=100,  # Optional
      max_embedding_requests_per_min=900,  # Optional
  )
  return rag_corpus, rag

#-------------------------------------------------

def retrieve_manual(rag_corpus, rag, query):
  """Manual search for Daou office """

  # Direct context retrieval
  responses = rag.retrieval_query(
      rag_resources=[
          rag.RagResource(
              rag_corpus=rag_corpus.name,
              # Optional: supply IDs from `rag.list_files()`.
              # rag_file_ids=["rag-file-1", "rag-file-2", ...],
          )
      ],
      text= query,
      similarity_top_k=10,  # Optional
      vector_distance_threshold=0.5,  # Optional
  )

  return responses



In [ ]:
display_name = "Daou_manual"

paths = ["gs://daou_office_manual/manual_org/DaouOffice 클라우드 관리자 가이드.pdf",
        "gs://daou_office_manual/manual_org/DaouOffice 클라우드 서비스 가이드 3.5.21.pdf",
        ]

rag_corpus, rag = create_rag_corpus(display_name, paths)

print(rag_corpus)
print(rag)


In [ ]:

query = "대쉬보드란 무엇인가요?"

responses = retrieve_manual(rag_corpus, rag, query)

for context in responses.contexts.contexts:
    # print(context)

    print(context.text)
    print(context.distance)

    print('=====')


1. 사이트 어드민의 [메뉴관리>홈 > 대시보드]을 클릭합니다.
2. 운영자 추가를 클릭합니다.
3. 조직도에서 운영자로 지정할 사용자를 모두 선택합니다.
4. 설정이 완료되면 저장을 클릭합니다.
[기본관리>메뉴 운영 권한 관리]에서도 대시보드 운영자를 설정할 수 있습니다.116 | 메뉴 관리
조직도 기능을 사용하지 않는다면, 사용자를 검색하여 추가할 수 있습니다.
대시보드에 대한 좀 더 자세한 설명이나 대시보드에 가젯을 설정하는 방법은 사용자 가이드를 참고하시기
바랍니다.
가젯 공개 설정
그림 6-1 가젯 공개 설정
사용자가 개인 대시보드에서 사용할 수 있는 가젯을 설정합니다. 공개한 가젯만 개인 대시보드에서 사용할 수 있습니
다.전자결재 | 117
1. 사이트 어드민의 [메뉴관리>홈 > 대시보드]을 클릭합니다.
2. 가젯 목록에서 공개, 비공개를 선택합니다.
3. 설정이 완료되면 저장을 클릭합니다.
6.2 전자결재
전자결재란 온라인에서 결재 문서를 작성하고, 결재를 할 수 있는 기능입니다.
0.25556055230334873
=====
결재선은 반드시 입력해야 하는 항목입니다.
■ 텍스트 - 문자를 한 줄로 입력할 수 있는 창입니다.
■ 멀티텍스트 - 문자를 여러 줄로 입력할 수 있는 창입니다.
■ 편집기 - 문자를 입력하거나 서식을 설정할 수 있는 도구입니다. 편집기를 넣지 않으면, 기안자가 글
자를 입력할 수 없습니다. 문자 입력이 필요한 경우에는 반드시 편집기를 넣어야 합니다.
■ 숫자 - 숫자를 입력할 수 있는 창입니다.
■ 통화 - 통화를 입력할 수 있는 창입니다.
■ 단일선택 - 여러 개의 항목 중에 하나를 선택할 수 있는 버튼입니다. 단일 선택을 클릭하면 라디오 버
튼이 삽입됩니다. 컴포넌트를 선택하고 오른쪽에서 A와 B 대신 선택 항목을 입력합니다.
■ 드롭박스 - 여러 개의 항목에서 하나를 선택할 수 있는 버튼입니다.
■ 복수 선택(Check) - 여러 개의 항목에서 하나 이상을 선택할 수 있는 버튼입니다. 복수 선택을 클릭하
면 체크박스가

## Integrate with Reasoning engine

### Helper methods

In [ ]:
# Initialize session history
store = {}

def get_session_history(session_id: str):
  """
    Get session history
  """
  if session_id not in store:
      store[session_id] = ChatMessageHistory()
  return store[session_id]

#---------------------------------------------------------------
def interactive_chat(agent, query, session_id):
  """
  Chat with agent with chat history.
  """
  respone = agent.query(input=query,
                        config={"configurable": {"session_id": session_id}},
  )
  return respone['output']

In [ ]:
# Create a RAG retrieval tool
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=rag_corpus.name,  # Currently only 1 corpus is allowed.
                    # Optional: supply IDs from `rag.list_files()`.
                    # rag_file_ids=["rag-file-1", "rag-file-2", ...],
                )
            ],
            similarity_top_k=10,  # Optional
            vector_distance_threshold=0.5,  # Optional
        ),
    )
)

In [ ]:
from googleapiclient import discovery
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain.memory import ChatMessageHistory
from langchain_core import prompts


# Define prompt template
prompt = {
    "history": lambda x: x["history"],
    "input": lambda x: x["input"],
    "agent_scratchpad": (lambda x: format_to_tool_messages(x["intermediate_steps"])),
} | prompts.ChatPromptTemplate.from_messages(
    [
        prompts.MessagesPlaceholder(variable_name="history"),
        ("user", "{input}"),
        prompts.MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [ ]:
from vertexai.preview import reasoning_engines

agent = reasoning_engines.LangchainAgent(
    prompt=prompt,
    model=MODEL,
    chat_history=get_session_history,
    model_kwargs={"temperature": 0},
    tools=[rag_retrieval_tool],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [ ]:
from time import perf_counter

session_id = "sess_101"

while True:
  query = input('사용자: ')

  if query == '종료': break

  t1_start = perf_counter()
  output = interactive_chat(agent, query, session_id)
  t1_end = perf_counter()

  display(Markdown(f"{t1_end-t1_start}"))
  display(Markdown(f"AI Agent : {output}"))
  print(f"------------------------------------ ")

chat_history = get_session_history(session_id)
chat_history.clear()
print(chat_history.messages)